In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')  # Set whitegrid style from Seaborn
#import mplfinance as mpf
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
import matplotlib.ticker as tkr
import matplotlib.dates as mdates
from pathlib import Path

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from collections import defaultdict
from scipy import stats

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)

# Keras
#from keras.preprocessing.text import one_hot
#from keras.preprocessing.text import Tokenizer
#Keras
import tensorflow as tf

# keras keras

from tensorflow.python.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Embedding
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.layers import Reshape, Dropout, Dense,Multiply, Dot, Concatenate,Embedding
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences
from keras.models import Sequential
#from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN, Dropout, GRU, Bidirectional,Dense
from keras.layers import GlobalMaxPooling1D
from keras.models import Model, Sequential
from keras.layers import Embedding, LSTM,Dense
from keras.optimizers import SGD
from keras.layers import Input
from keras.layers import concatenate
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Time-series
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# Modelling and Forecasting
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

#KERAS GRID Search
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.activations import swish
#
import math
#
#Seasonality, decompsition
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error
#
from datetime import datetime
from datetime import timedelta
#


### FUNCTION DEFINITION

In [ ]:
#Function to separate the DATA for LSTM
def createXY(dataset, n_lags):

    dataX, dataY = [], []

    for i in range(n_lags, len(dataset)):
            dataX.append(dataset[i - n_lags:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])

    return np.array(dataX),np.array(dataY)

In [ ]:
# Define the evaluation function
def evaluation(y_test, y_pred, data_test,title):
    # Plot the actual vs predicted values
    fig = plt.figure(figsize=(8, 3))
    plt.plot(y_test, color='red', label='Actual')
    plt.plot(y_pred, color='blue', label='Predicted')
    #plt.xticks(np.arange(0, len(df_ai_test), 100), df_ai_test.index[np.arange(0, len(df_ai_test), 100)], rotation=45)
    plt.title(f"Predictions vs Actual Data using {title}")
    #plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('EUR')
    plt.legend()
    plt.grid(False)

    # Assuming df_ai_test.index is a datetime index
    plt.xticks(np.arange(0, len(data_test), 100), data_test.index[np.arange(0, len(data_test), 100)].strftime('%Y-%m-%d'), rotation=45)

    # Use DateFormatter to format the x-axis to show only date
    #plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    #plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=20))  # Adjust the interval as needed
    # Calculate metrics
    #mape = mean_absolute_percentage_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    me = mean_error(y_test, y_pred)
    mda = mean_directional_accuracy(y_test, y_pred)
    da  = directional_accuracy(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Round metrics for display
   # mape = np.round(mape, 3)
    mae = np.round(mae, 3)
    mse = np.round(mse, 3)
    rmse = np.round(rmse, 3)
    me = np.round(me, 3)
    mda = np.round(mda, 3)
    da= np.round(da, 3)
    r2 = np.round(r2, 3)

    # Print metrics
    print('MAE: %f  --  MSE: %f  --  RMSE: %f  --  ME: %f  --  MDA: %f  --  DA: %f  --  R2: %f' % 
          (mae, mse, rmse, me, mda, da, r2))

    return mae, mse, rmse, me, mda, da, r2